In [18]:
import requests as r
from bs4 import BeautifulSoup

URL = "https://tw.news.yahoo.com/archive/{}.html"

for page in range(1, 11): # [1,2,3,4,5....10]
#page = 1
    print "Crawling page {}".format(URL.format(page))
    res = r.get(URL.format(page)) # res is a Result Object
    print "result is {}".format(res)
    #print res.text
    soup = BeautifulSoup(res.text, 'lxml')
    articles = soup.select('div.txt')
    #print len(soup.select('div.txt'))
    for article in articles:
        try:
            article_dict = {}
            article_dict['title'] = article.select_one('a').text
            cite = article.select_one('cite').text
            article_dict['source']  = cite.split(' - ')[0]
            article_dict['dt']  = cite.split(' - ')[1]
            for k in article_dict:
                print article_dict[k]
        except Exception as e:
            article_dict['source']  = ""
            article_dict['dt']  = ""
            print article
            print e



Crawling page https://tw.news.yahoo.com/archive/1.html
result is <Response [200]>
民報
2016年11月2日 上午9:29
​對決態勢升高！高醫大工會今正式對董事會提起刑事訴訟
MYDAILY
2016年11月2日 上午9:26
 泫雅等韓國藝人出席服裝品牌宣傳活動
健康醫療網
2016年11月2日 上午9:20
腸癌晚期轉移肝肺 他靠中醫溫腸化濁得救
今日新聞NOWnews
2016年11月2日 上午9:19
MLB／國民隊Harper不會參加經典賽
Yahoo奇摩新聞
2016年11月2日 上午9:17
閨蜜醜聞正炙 南韓提名新總理及財長
風傳媒
2016年11月2日 上午9:16
蔬果交易量占全國超過1/4 台北農產運銷「喊水會結凍」
風傳媒
2016年11月2日 上午9:16
張榮味家族掌控果菜批發上游通路 農會、農聯社、養豬協會全都包
今日新聞NOWnews
2016年11月2日 上午9:14
麵包師傅拍阿扁遭恐嚇 高市警：很快會給真相
全星網
2016年11月2日 上午9:11
李鍾碩客串《舉重妖精》 為李圣經新劇助力
鉅亨網
2016年11月2日 上午9:10
新漢攜手固高簽約設合資公司 布局工業4.0商機
Yahoo奇摩新聞
2016年11月2日 上午9:09
海龜搭高鐵 「海龜包廂」很特別
全星網
2016年11月2日 上午9:07
姜虎東任新綜MC 《我手中的父母》企劃12月開播
民視
2016年11月2日 上午9:05
騎士疑自摔滑行 衝撞超商物流車亡
今日新聞NOWnews
2016年11月2日 上午9:03
騎士疑車速過快 自摔滑行30公尺撞車爆頭亡
中央社
2016年11月2日 上午9:03
新台幣開盤貶 0.2分
中央社
2016年11月2日 上午9:03
美科技界奧斯卡 台資策會兩項入圍
鉅亨網
2016年11月2日 上午9:03
滿60歲、年資15年退休金可一次領 2020年起適用
華人健康網
2016年11月2日 上午9:02
打籃球吃蘿蔔乾！SOP處理五步驟快筆記
中央社
2016年11月2日 上午9:01
台股開盤跌42.47點
風傳媒
2016年11月2日 上午9:01
閻紀宇專欄：「十月驚奇」終於引爆 希拉蕊「電郵門」魔咒咎由自取
風傳媒
2

## 政府採購網

In [8]:
def str_to_dict(mystr):
    res_dict = {}
    for line in mystr.split("\n"):
        if len(line) != 0:
            #print line
            k = line.split(":")[0]
            v = line.split(":")[1]
            res_dict[k] = v
    return res_dict

In [65]:
import requests
from bs4 import BeautifulSoup

URL = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance"

raw_form = '''
method:search
searchMethod:true
searchTarget:ATM
orgName:
orgId:
hid_1:1
tenderName:
tenderId:
tenderStatus:4,5,21,29
tenderWay:
awardAnnounceStartDate:105/11/02
awardAnnounceEndDate:105/11/02
proctrgCate:
tenderRange:
minBudget:
maxBudget:
item:
hid_2:1
gottenVendorName:
gottenVendorId:
hid_3:1
submitVendorName:
submitVendorId:
location:
priorityCate:
isReConstruct:
btnQuery:查詢
'''

data = str_to_dict(raw_form)

r = requests.Session() # initialize a Session object

res = r.post(URL, data=data)
print res
soup = BeautifulSoup(res.text, 'lxml')

# Check total page number
total_page = int(soup.select_one('span.T11b').text) / 100 + 1
print total_page


# Page table
rows = soup.select_one('#print_area').select('tr')
for idx, row in enumerate(rows):
    if idx != 0 and idx != 101:
        cols = row.select('td')

        serial = cols[0].text
        org_name = cols[1].text
        prj_id = cols[2].text.split()[0]
        prj_name = cols[2].select_one('u').text
        prj_url = 'http://web.pcc.gov.tw/tps/' + cols[2].select_one('a')['href'][3:]
        print serial
        print org_name
        print prj_id
        print prj_name
        print prj_url

for page in range(2, total_page + 1):
    page_url = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex={}".format(page)
    print "crawling {}".format(page_url)
    res = r.get(page_url)
    print res
    soup = BeautifulSoup(res.text, 'lxml')
    # Page table
    rows = soup.select_one('#print_area').select('tr')
    for idx, row in enumerate(rows):
        if idx != 0 and idx != 101:
            cols = row.select('td')

            serial = cols[0].text
            org_name = cols[1].text
            prj_id = cols[2].text.split()[0]
            prj_name = cols[2].select_one('u').text
            prj_url = 'http://web.pcc.gov.tw/tps/' + cols[2].select_one('a')['href'][3:]
            print serial
            print org_name
            print prj_id
            print prj_name
            print prj_url

<Response [200]>
7
1 
新北市政府地政局
1020906-1-1
地政資訊網路e點通服務系統委託整合建置暨營運案(後續擴充)
http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51956439&tenderCaseNo=1020906-1-1
2 
澎湖縣政府
SV-10512
行政資訊設備效能改善暨提升－平板電腦採購案
http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51959629&tenderCaseNo=SV-10512
3 
財團法人中華民國對外貿易發展協會
10523002863
105年度「臺灣產業形象廣宣計畫」台灣精品標誌全球宣傳標語暨製拍台灣精品標誌宣傳影片招標案
http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51962333&tenderCaseNo=10523002863
4 
台灣中油股份有限公司
U8D05R017
Spare Parts for John Crane dry gas seal 1 Lot
http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do?newEdit=false&searchMode=common&method=inquiryForPublic&pkAtmMain=51962382&tenderCaseNo=U8D05R017
5 
臺北市政府衛生局
105MN18L
105年度失智症個案管理關懷服務計畫
http://web.pcc.gov.tw/tps/main/pms/tps/atm/atmAwardAction.do

IndexError: list index out of range

# 華航

In [4]:
import requests
from bs4 import BeautifulSoup

URL = "https://www.china-airlines.com/tw/en/FlightSearchResults"

data1 = '''
{"CorporateLoggedIn":false,"PassengerDetail":{"Adults":1,"Students":"0","Children":0,"Infants":0},"DepartureDate":"201611280000","ReturnDate":"201611300000","DepartureStn":"TPE","ArrivalStn":"NRT","TripType":"R","TravelClass":"Y","RestrictionsPromoCode":"","FlightSearchPageType":"BasicFlightSearch"}

'''

myheader = {'Accept': 'application/json, text/plain, */*',
 'Accept-Encoding': 'gzip, deflate, br',
 'Accept-Language': 'en-US,en;q=0.8',
 'Cache-Control': 'no-cache',
 'Connection': 'keep-alive',
 'Content-Length': '300',
 'Content-Type': 'application/json;charset=UTF-8',
 'Cookie': '__utmt_UA-46599335-8=1; _dc_gtm_UA-63072599-7=1; _gat_UA-46599335-1=1; _gat_UA-54880298-3=1; cookie-policy=1; country=%E5%8F%B0%E7%81%A3; ASP.NET_SessionId=44g41c1kq1n4qbqnmfk1d5es; TAFSessionId=tridion_4a6eb2df-e689-45b3-a4e9-8900559d1fd2; TAFTrackingId=tridion_eb7ac7c2-e622-42ec-a7ef-f96c4b8d2ba0; G_ENABLED_IDPS=google; dtLatC=44; dtPC=-; dtCookie=ACC0C1815272CB522D847BB8774EA9E7|T0xCTnwx; TDSCAL2016=USER_ID=182231412; __utma=225761605.1236774433.1478035933.1478035933.1478035933.1; __utmb=225761605.3.10.1478035933; __utmc=225761605; __utmz=225761605.1478035933.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); _ga=GA1.2.1236774433.1478035933',
 'Host': 'www.china-airlines.com',
 'Origin': 'https',
 'Pragma': 'no-cache',
 'Referer': 'https',
 'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13B143 Safari/601.1'}

#print str_to_dict(header1_str)

r = requests.Session()

res = r.post(URL, data=data1, headers=myheader)

In [5]:
res.text

u'{"member":{"DFPToken":null,"DFPNo":null,"SessionID":null},"CorporateOrVoucherCode":"","PostUrl":"https://bookingportal.china-airlines.com/eRetailPortal/Mobile.svc/Mobile/Search","ErrorMessages":[],"CorporateLoginExpired":false,"CorporateLoginExpiredMsg":null,"IsMobileorTablet":true}'

In [10]:
data_raw = '''
B_DATE_1:201611280000
B_DATE_2:201611300000
B_LOCATION_1:TPE
E_LOCATION_1:NRT
B_LOCATION_2:NRT
E_LOCATION_2:TPE
EBA:TW
SessionID:kq0mw3c13xulwcxbkyrjogva
STUDENTS:0
ADULTS:1
CHILDS:0
INFANTS:0
DFPNo:
DFPToken:
LANG:GB
Token:
returnURL:http://www.china-airlines.com/tw/en
TRIP_TYPE:R
CABIN:Y
'''

header_raw = '''
Accept:text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8
Accept-Encoding:gzip, deflate, br
Accept-Language:en-US,en;q=0.8
Cache-Control:no-cache
Connection:keep-alive
Content-Length:300
Content-Type:application/x-www-form-urlencoded
Cookie:cookie-policy=1; country=%E5%8F%B0%E7%81%A3; ASP.NET_SessionId=1ssrwp50eqbpvgoqpglyq5tl; dtPC=-; dtCookie=ACC0C1815272CB522D847BB8774EA9E7|T0xCTnwx; dtLatC=4; _ga=GA1.3.1236774433.1478035933; __utmt_track0=1; __utmt_track1=1; _gat_UA-46599335-1=1; __utmt_UA-46599335-8=1; __utma=225761605.1236774433.1478035933.1478035933.1478053815.2; __utmb=225761605.7.10.1478053815; __utmc=225761605; __utmz=225761605.1478035933.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); _dc_gtm_UA-63072599-7=1; _ga=GA1.2.1236774433.1478035933; _gat_UA-54880298-3=1
Host:bookingportal.china-airlines.com
Origin:https://www.china-airlines.com
Pragma:no-cache
Referer:https://www.china-airlines.com/tw/en
Upgrade-Insecure-Requests:1
User-Agent:Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13B143 Safari/601.1
'''

header = str_to_dict(header_raw)
data = str_to_dict(data_raw)
print header


res = r.post("https://bookingportal.china-airlines.com/eRetailPortal/Mobile.svc/Mobile/Search", data=data, headers=header)

{'Origin': 'https', 'Content-Length': '300', 'Accept-Language': 'en-US,en;q=0.8', 'Accept-Encoding': 'gzip, deflate, br', 'Host': 'bookingportal.china-airlines.com', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8', 'Upgrade-Insecure-Requests': '1', 'Connection': 'keep-alive', 'Cookie': 'cookie-policy=1; country=%E5%8F%B0%E7%81%A3; ASP.NET_SessionId=1ssrwp50eqbpvgoqpglyq5tl; dtPC=-; dtCookie=ACC0C1815272CB522D847BB8774EA9E7|T0xCTnwx; dtLatC=4; _ga=GA1.3.1236774433.1478035933; __utmt_track0=1; __utmt_track1=1; _gat_UA-46599335-1=1; __utmt_UA-46599335-8=1; __utma=225761605.1236774433.1478035933.1478035933.1478053815.2; __utmb=225761605.7.10.1478053815; __utmc=225761605; __utmz=225761605.1478035933.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); _dc_gtm_UA-63072599-7=1; _ga=GA1.2.1236774433.1478035933; _gat_UA-54880298-3=1', 'Pragma': 'no-cache', 'Cache-Control': 'no-cache', 'Referer': 'https', 'User-Agent': 'Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 

In [11]:
print res.text

<html><head><meta name='HandheldFriendly' content='true' /><meta name='format-detection' content='telephone=no' /><meta name='apple-mobile-web-app-capable' content='yes'><meta name='apple-mobile-web-app-status-bar-style' content='default' /><meta http-equiv="Cache-Control" content="no-store" /><meta http-equiv="Pragma" content="no-cache" /><meta http-equiv="Cache-Control" content="no-cache" /><title>China Airlines</title><script type="text/javascript" src="/dtagent637_bjq_1007.js" data-dtconfig="rid=RID_-1048102435|rpid=1835876920|domain=china-airlines.com|tp=500,50,0|bandwidth=300|reportUrl=dynaTraceMonitor"></script><link rel="stylesheet" href="https://book.china-airlines.com/plnext/default/Merci_31.1.17_211016/static/merciAT/modules/common/css/merci.css"><link rel="stylesheet" href="https://book.china-airlines.com/plnext/mobile4CI/201610130750/static/css/custom_merci.css"></head><body id='bcal'><div id='top' class=' xPrintAdaptX xPrintAdaptY'><div class='xTplContent xPrintAdaptX xPr

In [25]:
# request # 3
soup3 = BeautifulSoup(res.text, 'lxml')
URL3 = soup3.find('form')['action']
ENC = soup3.find('input')['value']
data3 = {}
for ipt in soup3.find_all('input'):
    #print ipt['name']
    data3[ipt['name']] = ipt['value']

res3 = r.post(URL3, data=data3, headers=header)

In [27]:
print res3.text

<!DOCTYPE html>
<html dir="ltr" lang="en"> <head> <meta name="HandheldFriendly" content="true" /> <meta name="format-detection" content="telephone=no" /> <meta name="apple-mobile-web-app-capable" content="yes"> <meta name="apple-mobile-web-app-status-bar-style" content="default" /> <meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1"> <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0" /> <link rel="apple-touch-icon" href='https://bookingportal.china-airlines.com/plnext/mobile4CI/201610130750/static/img/client/airline_ico.png'> <link rel='icon' type='image/png' href='https://bookingportal.china-airlines.com/plnext/mobile4CI/201610130750/static/img/client/airline_ico.png'/> <link rel="apple-touch-icon-precomposed" href='https://bookingportal.china-airlines.com/plnext/mobile4CI/201610130750/static/img/client/airline_ico.png'> <link href='https://bookingportal.china-airlines.com/plnext/default/Merci_31.1.20_271016/static/merciAT/modu

In [28]:
soup3 = BeautifulSoup(res3.text, 'lxml')

In [40]:
json_str = soup3.find_all('script')[6].text.split('json: ')[1].split(' }); ')[0]

In [41]:
import json
json.loads(json_str)

{u'data': {u'booking': {u'MCAL_A': {u'globalList': {u'currencyCode': [[u'AFN',
       u'Afghani'],
      [u'ALL', u'Albania Lek'],
      [u'DZD', u'Algerian Dinar'],
      [u'ANG', u'Antille Guilder'],
      [u'ARS', u'Argentin Peso'],
      [u'AWG', u'Aruban Florin'],
      [u'AUD', u'Australian Dollar'],
      [u'AZN', u'Azerbaijan Manat'],
      [u'BSD', u'Bahamian Dollar'],
      [u'BHD', u'Bahraini Dinar'],
      [u'BDT', u'Bangladesh Taka'],
      [u'BBD', u'Barbados Dollar'],
      [u'BYN', u'Belarusian Ruble'],
      [u'BZD', u'Belize Dollar'],
      [u'BMD', u'Bermuda Dollar'],
      [u'BTN', u'Bhutan Ngultrum'],
      [u'BOB', u'Boliviano'],
      [u'VEF', u'Bolivar Fuerte'],
      [u'BWP', u'Botswana Pula'],
      [u'GBP', u'British Pound'],
      [u'BND', u'Brunei Dollar'],
      [u'BGN', u'Bulgarian Lev'],
      [u'BIF', u'Burundi Franc'],
      [u'CAD', u'Canadian Dollar'],
      [u'CVE', u'Cape Verde Escudo'],
      [u'KYD', u'Cayman Islands Dollar'],
      [u'XAF', u'CF